In [ ]:
!pip install pycryptodome
!pip install libnum

     |████████████████████████████████| 13.7MB 308kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/CoLab/Applied Crytography/')
from corelib import *

#1. Chuẩn bị

##1.1. RSA
Bob dùng hàm rsa_keygen để tạo public key & private key

In [ ]:
publickey, privatekey = rsa_keygen()
print('Public key', publickey)
print('Private key', privatekey)

Public key (1261539080901866626210198941283, 772828713920824982523364279670053105891403889745831724382191)
Private key (485584346270657878094065684602587157349850049410375886266059, 772828713920824982523364279670053105891403889745831724382191)


Sau đó ghi public key vào file bob_publickey.txt rồi gửi cho Alice

In [ ]:
file_write_array('/content/drive/MyDrive/CoLab/Applied Crytography/bob_publickey.txt', publickey)

True

Bob cũng nhận được file alice_publickey.txt của Alice, Bob đọc file này để có public key

In [ ]:
alice_rsa_publickey = file_read_array('/content/drive/MyDrive/CoLab/Applied Crytography/alice_publickey.txt')
print(alice_rsa_publickey)

[1005338133771723824417952592417, 623404998140488331528930995460821027130628082907569713675721]


#2. Giải mã

##2.1. Giải mã AES key

Bob dùng hàm file_read_array để đọc AES Key đã mã hóa mà Alice gửi

In [ ]:
ctext = file_read_array('/content/drive/MyDrive/CoLab/Applied Crytography/alice_aes_key.txt')
print(ctext)

[203523798616596190259241860845778380016475912912773119951076, 396590001221089078513006320129893453293079366085233705100240]


Sau đó dùng hàm rsa_decrypt để giải mã chuỗi cipher để có được AES Key của Alice

In [ ]:
alice_aes_key = rsa_decrypt(ctext, privatekey)
print(alice_aes_key)

Alice@AES1234567


##2.2. Giải mã  thông điệp
Sau khi nhận được nội dung từ alice trong file alice_cipherfile.txt, Bob tiến hành giải mã thông điệp

In [ ]:
aes_decrypt(alice_aes_key.encode('utf8'), '/content/drive/MyDrive/CoLab/Applied Crytography/alice_cipherfile.txt', 'alice_plainmessage.txt')
plainfile = open('alice_plainmessage.txt')
message = plainfile.read()
print('Message', message)

Message Hello Bob! Can I borrow you some money?


##2.3. Xác thực chữ ký
Để kiểm chứng thông điệp trên đúng là do Alice gửi, Bob tiến hành xác thực chữ ký

In [ ]:
signature = file_read_array('/content/drive/MyDrive/CoLab/Applied Crytography/alice_signature.txt')
print(signature)

[283073230491058599787932000534120571550103144536462807238993, 517488592760781465185443581500235075515763541968393963429505, 412820013345258904145683428478892228242952202461386237742598, 546140119731087056310042770713775914250063023565398970871223, 575835242056158095723787222338958625392036342606605315394284, 382654049259113701774546052595256660378459446432292328379864, 428444943822098354262655154973874237007666455632825413253832]


Bob dùng rsa_decrypt để giải mã chữ ký số của Alice bằng chính public key của Alice để có được chuỗi digest từ chữ ký

In [ ]:
alice_digest = rsa_decrypt(signature, alice_rsa_publickey)
print('Signature digest', alice_digest)

Signature digest c32e23a6fc55db52d43bf13fd6eda0121fb03a3124bec81ea9405838793bc994


Bob tiếp tục tạo chuỗi digest từ chính message mà mình đã đọc được

In [ ]:
compare_digest = str(sha_digest(message.encode('utf8')))
print(compare_digest)

c32e23a6fc55db52d43bf13fd6eda0121fb03a3124bec81ea9405838793bc994


Cuối cùng, Bob compare chuỗi digest từ chữ ký với chuỗi digest từ message mà mình đọc được để xác thực nội dung

In [ ]:
if (alice_digest == compare_digest):
  print('The message is from Alice')
else:
  print('The message is NOT from Alice')

The message is from Alice
